In [19]:
import MeCab
import re
import os
from bs4 import BeautifulSoup
from urllib import request
import glob
from pathlib import Path

In [20]:
def get_author_id_dict(url="https://www.aozora.gr.jp/index_pages/person_all.html"):
    '''
    機能：著者名と著者IDの辞書を取得する関数
    引数：青空文庫の作家リストページのURL(省略可)
    返り値：著者名と著者IDの辞書
    '''
    response = request.urlopen(url)
    soup = BeautifulSoup(response)
    response.close()

    author_id = {}
    elms = soup.select('li a')
    for e in elms:
        href = e['href']
        idx = re.findall('person([0-9]*)\.html.*',href)
        author_id[e.text.replace(' ','')] = f'{int(idx[0]):0=6}'
    return author_id

author_id_dict = get_author_id_dict()

import pickle
with open('author_id_dict.pkl','wb') as f:
    pickle.dump(author_id_dict,f)

In [21]:
def get_local_author_id_dict():
    import pickle
    with open('author_id_dict.pkl','rb') as f:
        author_id_dict = pickle.load(f)
    return author_id_dict
get_local_author_id_dict()

{'アーヴィングワシントン': '001257',
 'アークムフレデリック': '001657',
 '愛知敬一': '001234',
 '会津八一': '001245',
 'アインシュタインアルベルト': '001428',
 '饗庭篁村': '000374',
 '青木栄瞳': '000330',
 '青木正児': '002091',
 '青空文庫': '001790',
 '青野季吉': '001542',
 '青柳喜兵衛': '001314',
 '秋田雨雀': '001584',
 '秋田滋': '001162',
 '秋月種樹': '001331',
 '芥川紗織': '001990',
 '間所紗織': '002085',
 '芥川多加志': '002132',
 '芥川竜之介': '000879',
 '浅井洌': '000380',
 '朝倉克彦': '000139',
 '浅沼稲次郎': '001487',
 '浅野正恭': '001816',
 '浅野和三郎': '001034',
 '東健而': '001465',
 '阿部次郎': '001421',
 '阿部徳蔵': '000225',
 '安倍能成': '001909',
 'アポリネールギヨーム': '002135',
 '天川恵三郎': '001964',
 'アミーチスエドモンド・デ': '001048',
 '鮎川義介': '001948',
 '新井紀一': '001902',
 '荒木光二郎': '001839',
 'アラルコンペドロ・アントニオ': '001716',
 '有島武郎': '000025',
 'アルチバシェッフミハイル・ペトローヴィチ': '000366',
 'アルテンベルクペーター': '001190',
 'アレニウススヴァンテ': '000226',
 'アレンジェームズ': '001842',
 'アレンリリー・Ｌ': '001843',
 '淡島寒月': '000388',
 '安西冬衛': '001863',
 'アンデルセンハンス・クリスチャン': '000019',
 '安藤盛': '001791',
 'アンドレーエフレオニード・ニコラーエヴィチ': '000907',
 '李光洙': '001601',
 '李箱': '0015

In [22]:
def text_pre_processing(txt):# 本文の前処理
    '''
    機能：文字列(テキスト)の不要な部分(本文以外の文書や「」など)を削除する関数
    変数：前処理したい文字列
    返り値：処理した文字列
    '''
    txt = txt.split('底本')[0]
    txt = txt.replace('｜','')
    txt = txt.replace('／','').replace('＼','').replace('―','')
    txt = re.sub('《.*?》','', txt)
    txt = re.sub('［＃中見出し］.*?［＃中見出し終わり］','', txt)
    txt = re.sub('［＃.*?］','', txt)
    txt = txt.replace('「', '').replace('」', '').replace('『','').replace('』','')
    txt = txt.replace('\r','').replace('\n','').replace('\u3000', '')
    txt = re.sub('([。！？])', r'\1\n', txt)
    return txt

def make_alltext_of_specified_author(author):
    '''
    機能：指定した著者名の全作品を文字列として結合し、ファイルに保存する
    変数：著者名を名字と名前を続く形で与える ex)「夏目漱石」「太宰治」
    返り値：結合した文字列(著者の全作品)
    '''
    
    all_filename_list = glob.glob(f'/Users/akimotokazuki/cards/{author_id_dict[author]}'+r'/files/*/*.txt')
    file_all_content = ''
    file_stem_list = []
    for file in all_filename_list:
        file_id = Path(file).stem.split('_')[0]
        with open(file,'rb') as f:
            file_content = f.read().decode('shiftjisx0213')
        try:
            file_content = re.split('-{10,}',file_content)[2]
            if  file_id in file_stem_list:
                continue
            file_content = text_pre_processing(file_content)
            file_all_content += file_content
            file_stem_list.append(file_id)
        except Exception as e: # 例外処理。置換に失敗したファイルの内容は捨てる
            pass
        
    with open(f'word2vec_data/Git{author}.txt','w') as f:
        print(file_all_content, file = f)   
    
    return file_all_content

def create_segmented_list_of_sentences_by_author(text_data,stopword=True): 
    '''
    機能：文章をわかち書きし、Stopword(「の」や「が」など)を文章から省く関数
    変数：処理したい文字列、定義したストップワード
    返り値：処理した文字列
    '''
    import re
    import MeCab as mc
    
    dic_neo = ' -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd' 
    wakati = mc.Tagger("-Owakati"+dic_neo)
    
    if stopword:
        stopwords = create_japanese_stopword()
    else:
        stopwords = []
    
    text_wakati_data = [wakati.parse(i).split() for i in text_data.split('\n')]
    text_data_result = []
    
    for s in text_wakati_data:
        s_list = []
        for w in s:
            if w not in stopwords:
                s_list.append(w)
        text_data_result.append(s_list)
                    
    return text_data_result

In [23]:
# 太宰治の単語データを取得
alltext_of_author = make_alltext_of_specified_author("太宰治")
text_data_result = create_segmented_list_of_sentences_by_author(alltext_of_author, stopword=False)
print(len(text_data_result)) # 行数

80398


In [26]:
uniq_words = list(set(word for sentence in text_data_result for word in sentence))
print(len(uniq_words)) # ユニーク単語

46106


In [34]:
# 訓練データ
train_file_content = ""
for row in text_data_result[:int(len(text_data_result)/5*3.5)]:
        for word in row:
            train_file_content += word + " "
        train_file_content += '\n'
print(train_file_content[:100])
print(len(train_file_content.split(" ")))

所謂 社会主義 の 世の中 に なる の は 、 それ は 当り前 の 事 と 思わ なけれ ば なら ぬ 。 
民主 々 義 と は 云っ て も 、 それ は 社会 民主 々 義 の 事 で あっ
1098108


In [35]:
# テストデータ
test_file_content = ""
for row in text_data_result[int(len(text_data_result)/5*3.5):int(len(text_data_result)/5*4.2)]:
        for word in row:
            test_file_content += word + " "
        test_file_content += '\n'
print(test_file_content[:100])
print(len(test_file_content.split(" ")))

あまりに 静か に 、 早く 息 を ひきとっ た から で ござい ましょ う 。 
けれども 、 私 は 、 その とき 驚か なかっ た 。 
何もかも 神さま の 、 おぼしめし と 信じ て
225486


In [36]:
# 評価データ
valid_file_content = ""
for row in text_data_result[int(len(text_data_result)/5*4.2):]:
        for word in row:
            valid_file_content += word + " "
        valid_file_content += '\n'
print(valid_file_content[:100])
print(len(valid_file_content.split(" ")))

におい 山路 は 、 冥土 に 在る 山 の 名前 かも 知れ ない 。 
かな は 、 女児 の 名 で あろ う 。 
消え まよい けり は 、 いかにも 若い女 の 幽霊 らしく 、 あ われ
254808


In [33]:
# わかち書きテキストファイル生成
train_file_name = 'train_dazai.txt'  
with open(train_file_name, 'w', encoding='utf-8') as f:
    f.write(train_file_content)

test_file_name = 'test_dazai.txt'  
with open(test_file_name, 'w', encoding='utf-8') as f:
    f.write(test_file_content)
    
valid_file_name = 'valid_dazai.txt' 
with open(valid_file_name, 'w', encoding='utf-8') as f:
    f.write(valid_file_content)

2504486

846617

836351